In [1]:
#Import dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from flask import Flask, render_template, redirect

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ljean\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Step 1 - Scraping Mars Websites

In [3]:
############ PART 1 - NASA MARS NEWS  ##############################
#Establish Mars News URL to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
response = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [4]:
#pull in results to enable easier parsing
# results = soup.find_all('div', class_='content_title')
# results
#note that first article headline is at index [1]

# article = soup.find_all('div', class_='article_teaser_body')
# article
#note that first article teaser is at index [0]

In [5]:
# Return the latest news title and paragraph stored as variables
news_title = soup.find_all('div', class_='content_title')[1].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text
print('----------------------------')
print(news_title)
print('------------------------------------------------------------------------')
print(news_p)
print('----------------------------')

----------------------------
New Study Challenges Long-Held Theory of Fate of Mars' Water
------------------------------------------------------------------------
The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.
----------------------------


In [6]:
############ PART 2 - JPL Mars Space Images  ##############################
#Use splinter to navigate the JPL Mars site and find the image url for the current Featured Mars Image 

#establish splinter base url link
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)
jpm_html = browser.html
response = requests.get(jpl_url)

# Create BeautifulSoup object; parse with 'html.parser'
soup_image = bs(jpm_html, 'html.parser')

In [7]:
#Find the image url to the full size .jpg image.
image = soup_image.find_all('img', class_='headerimage fade-in')[0]['src']
image

'image/featured/mars3.jpg'

In [8]:
#clean existing URL to avoid code errors
cleaned_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

#assign the url string to a variable called featured_image_url.
featured_image_url = cleaned_url + image
print(featured_image_url)


https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg


In [9]:
############ PART 3 - Mars Facts  ##############################
#Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
#Then convert data to a HTML table string.

In [10]:
#Access the mars facts url
facts_url = 'https://space-facts.com/mars/'

In [11]:
#Load the url in using pandas
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
#Convert table into a dataframe displaying facts about the planet including Diameter, Mass, etc.
facts_df = tables[0]
facts_df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [13]:
#Rename columns
facts_df.columns = ["Description", "Value"]
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [14]:
#convert dataframe back to html table string
html_table = facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [15]:
#Save a copy of the html
facts_df.to_html('table.html')

In [16]:
############ PART 4 - Mars Hemispheres  ##############################
#Click on link to each hemisphere to retrive the image

In [17]:
#Capture the usgs astrogeology website
geo_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
image_url = 'https://astrogeology.usgs.gov'

In [18]:
#Establish splinter connection
browser.visit(geo_url)
geo_html = browser.html
response = requests.get(geo_url)

# Create BeautifulSoup object; parse with 'html.parser'
soup_geo = bs(geo_html, 'html.parser')

In [19]:
# Retrieve initial elements that contain hemisphere information
items = soup_geo.find('div', class_='collapsible results')
images = items.find_all('div', class_='item')

In [20]:
#create placeholder hemisphere list
mars_hemispheres = []

#Create loop to move through each of the web pages
for image in images:
    
    #collect title information
    hemisphere = image.find('div', class_="description")
    title = hemisphere.h3.text.strip()
    
    #collect link information which clicks to the next website where full resolution available
    link = hemisphere.a['href']
    browser.visit(image_url + link)     
    
    #collect links from second website for full resolition image
    img_html = browser.html
    img_soup = bs(img_html, 'html.parser')
    
    img_link = img_soup.find('div', class_='downloads')
    img_url= img_link.find('li').a['href']
    
    #Create dictionary to store title and image data
    hemisphere_dict = {}
    hemisphere_dict['title'] = title
    hemisphere_dict['img_url'] = img_url
    
    mars_hemispheres.append(hemisphere_dict)

mars_hemispheres  



[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

Step 2 - MongoDB and Flask Application

In [21]:
#Create a dictionary of all above scraped data to facilitate coversion
mars_data_dict = {
    "News_Headlines": news_title,
    "News_Tagline": news_p,
    "Featured_Image": featured_image_url,
    "Mars_Facts":facts_df,
    "Mars_Hemispheres": mars_hemispheres}

In [22]:
mars_data_dict

{'News_Headlines': "New Study Challenges Long-Held Theory of Fate of Mars' Water",
 'News_Tagline': 'The new science results indicate that a large quantity of the Red Planet’s water is trapped in its crust rather than having escaped into space.',
 'Featured_Image': 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg',
 'Mars_Facts':             Description                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
 'Mars_Hemispheres': [{'title': 'Cerberus Hemisphere Enhanced',
   'img_url